<a href="https://colab.research.google.com/github/Nerujan/Colab/blob/main/XLM_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install datasets


from datasets import load_dataset, Dataset, DatasetDict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Define the unwanted labels you want to map to "O"
unwanted_labels = ["B-LOC", "I-LOC", "B-MISC", "I-MISC"]

# Define your label map
label_map = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-COMM": 5,
    "I-COMM": 6
}

# Load your dataset (replace with your actual dataset path)
dataset_path = "/content/drive/MyDrive/Dataset/Final_Tamil.txt"
dataset = load_dataset("text", data_files=dataset_path)

# Function to parse text and extract tokens with labels
def parse_text(example):
    words, labels = [], []
    text_data = example.get("text", "").strip()

    if not text_data:
        return {"tokens": [], "ner_tags": []}

    # Assuming your data is formatted like "word1 O word2 B-PER ..."
    tokens = text_data.split(" ")
    i = 0
    while i < len(tokens):
        if i + 1 < len(tokens) and tokens[i + 1] in label_map:
            words.append(tokens[i])
            labels.append(label_map.get(tokens[i + 1], 0))  # Default to 'O'
            i += 2
        else:
            words.append(tokens[i])
            labels.append(label_map.get("O", 0))  # Default to 'O'
            i += 1

    return {"tokens": words, "ner_tags": labels}

# Apply parsing function
dataset = dataset.map(parse_text)

# Define the function to map unwanted labels to "O"
def map_unwanted_labels(example):
    # Update labels to "O" (0) if they are in unwanted_labels
    new_labels = []
    for label in example["ner_tags"]:
        # Get the label text from label_map and map unwanted labels to "O"
        label_text = list(label_map.keys())[list(label_map.values()).index(label)] if label != -100 else "O"
        if label_text in unwanted_labels:
            new_labels.append(label_map["O"])  # Map to O (0)
        else:
            new_labels.append(label)
    example["ner_tags"] = new_labels
    return example

# Apply the function to update the labels
dataset = dataset.map(map_unwanted_labels)



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/87898 [00:00<?, ? examples/s]

Map:   0%|          | 0/87898 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
import os

# Define the label mapping from the provided model

label_map = {
     "O": 0,  # Outside of a named entity
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-COMM": 5,
    "I-COMM": 6
}

id2label = {v: k for k, v in label_map.items() if v != -100}
label2id = {k: v for v, k in id2label.items()}
num_labels = len(set(label_map.values()))  # Count unique label indices
# Load pre-trained model with correct label mapping




In [ ]:
 # Load pre-trained model and tokenizer
model_name = "Lokeshwaran/xlm-roberta-base-fintuned-panx-ta-hi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Load dataset
dataset_path = "/content/drive/MyDrive/Dataset/Final_Tamil.txt"
dataset = load_dataset("text", data_files=dataset_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/988 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
# Function to parse text and extract tokens with labels
def parse_text(example):
    words, labels = [], []
    text_data = example.get("text", "").strip()

    if not text_data:
        return {"tokens": [], "ner_tags": []}

    tokens = text_data.split(" ")
    i = 0
    while i < len(tokens):
        if i + 1 < len(tokens) and tokens[i + 1] in label_map:
            words.append(tokens[i])
            labels.append(label_map.get(tokens[i + 1], 0))  # Default to 'O'
            i += 2
        else:
            words.append(tokens[i])
            labels.append(label_map.get("O", 0))
            i += 1

    return {"tokens": words, "ner_tags": labels}

# Apply parsing function
dataset = dataset.map(parse_text)

Map:   0%|          | 0/87898 [00:00<?, ? examples/s]

In [ ]:
# print(example["ner_tags"])

NameError: name 'example' is not defined

In [ ]:
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(
        batch["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    all_labels = []

    for i in range(len(batch["tokens"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        labels = []

        for word_idx in word_ids:
            if word_idx is None:
                labels.append(0)  # Special tokens should be ignored
            else:
                label = batch["ner_tags"][i][word_idx]
                labels.append(label)  # Assign 0 for unused labels

        labels += [0] * (len(tokenized_inputs["input_ids"][i]) - len(labels))
        all_labels.append(labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/87898 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Model/Checkpoints",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Model/Logs",
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False,
    resume_from_checkpoint=True
)


In [ ]:
print(next(iter(tokenized_datasets["train"]))["labels"])


In [ ]:
print(model.config.id2label)
print(model.config.label2id)


{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-COMM', 6: 'I-COMM'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-COMM': 5, 'I-COMM': 6}


In [ ]:
for example in tokenized_datasets["train"]:
    print(example["labels"])
    assert all(-100 <= l < num_labels for l in example["labels"]), "Invalid label detected!"
    break



In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Force synchronous error reporting


In [ ]:
for example in tokenized_datasets["train"]:
    assert all(-100 <= l < num_labels for l in example["labels"]), "Invalid label detected!"


In [ ]:
from transformers import TrainingArguments

checkpoint_path = "/content/drive/MyDrive/Model/Checkpoints/checkpoint-20000"  # Replace with your actual checkpoint path

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Model/Checkpoints",  # Your main output directory
    overwrite_output_dir=True,  # Overwrite if needed
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/Model/Logs",
    logging_steps=10,
    report_to="wandb",  # If using Weights & Biases
    resume_from_checkpoint=checkpoint_path  # Resume from the checkpoint
)


In [ ]:
from transformers import Trainer
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Resume training
trainer.train(resume_from_checkpoint=checkpoint_path)


<ipython-input-9-f8c4e17c0f64>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2020csc055 (2020csc055-university-of-jaffna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20010,0.004700
20020,0.001500
20030,0.003600
20040,0.002300
20050,0.001800
20060,0.002500
20070,0.003700
20080,0.005300
20090,0.001900
20100,0.001700


TrainOutput(global_step=32964, training_loss=0.0009830284043689408, metrics={'train_runtime': 3700.8674, 'train_samples_per_second': 71.252, 'train_steps_per_second': 8.907, 'total_flos': 1.7226375420519936e+16, 'train_loss': 0.0009830284043689408, 'epoch': 3.0})

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# Path to the latest checkpoint (update this with your actual checkpoint path)
checkpoint_path = "/content/drive/MyDrive/Model/Checkpoints/checkpoint-32964"  # Replace xxxxx with actual checkpoint number

# Load the trained model and tokenizer
model = AutoModelForTokenClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)


In [ ]:
# Create the NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")


Device set to use cpu


In [ ]:
text = "பௌத்தர்கள் புத்தர் போதனைகளை பின்பற்றுகின்றனர் "

# Get predictions
ner_results = ner_pipeline(text)

# Display results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")


Entity: பௌத்தர்கள், Label: COMM, Score: 0.9594
Entity: புத்தர், Label: COMM, Score: 0.9611
Entity: போதனைகளை, Label: COMM, Score: 0.7749
Entity: பின்பற்றுகின்றனர், Label: COMM, Score: 0.7958


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [ ]:
tokens = tokenizer(text, return_offsets_mapping=True)
print(tokens)


{'input_ids': [0, 8838, 113816, 3770, 41568, 15413, 3770, 2912, 197592, 9982, 6819, 235005, 120159, 2912, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 1), (1, 2), (2, 5), (5, 10), (11, 13), (13, 16), (16, 18), (19, 22), (22, 24), (24, 27), (28, 36), (36, 43), (43, 45), (0, 0)]}


In [ ]:
def merge_subwords(text, offset_mapping, predictions, scores):
    entities = []
    current_word = ""
    current_label = None
    current_score = 0
    subword_count = 0

    for i, (offset, pred, score) in enumerate(zip(offset_mapping, predictions, scores)):
        start, end = offset  # Get character offsets

        if start == 0 and end == 0:  # Ignore special tokens (CLS, SEP)
            continue

        word = text[start:end]  # Extract actual word from text
        label = id2label.get(pred, "O")  # Convert prediction to label

        if label.startswith("B-") or (label != "O" and label != current_label):
            if current_word:
                entities.append({"word": current_word, "label": current_label, "score": current_score / subword_count})

            current_word = word
            current_label = label
            current_score = score
            subword_count = 1
        elif label.startswith("I-") and current_label == label.replace("I-", "B-"):
            current_word += word
            current_score += score
            subword_count += 1
        else:
            if current_word:
                entities.append({"word": current_word, "label": current_label, "score": current_score / subword_count})

            current_word = ""
            current_label = None
            current_score = 0
            subword_count = 0

    if current_word:
        entities.append({"word": current_word, "label": current_label, "score": current_score / subword_count})

    return entities


In [ ]:
import torch

text = "முஸ்லிம்கள் இந்த நாட்டில் மிகவும் நல்ல வியாபாரிகள்"

# Tokenize with offset mapping
tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True)

# Store offset mapping separately before removing it
offset_mapping = tokens.pop("offset_mapping").squeeze(0).tolist()  # Get the correct offset mapping

# Move tensors to the same device as the model
tokens = {k: v.to(model.device) for k, v in tokens.items()}

# Get model predictions
outputs = model(**tokens)

# Process outputs
predictions = torch.argmax(outputs.logits, dim=-1).squeeze().tolist()
scores = torch.nn.functional.softmax(outputs.logits, dim=-1).max(dim=-1).values.squeeze().tolist()

# Merge subwords (pass `offset_mapping` instead of `tokens`)
ner_results = merge_subwords(text, offset_mapping, predictions, scores)

# Display results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['label']}, Score: {entity['score']:.4f}")

Entity: முஸ்லிம், Label: B-COMM, Score: 0.9044
Entity: கள், Label: B-COMM, Score: 0.8840
Entity: மிகவும், Label: B-COMM, Score: 0.5641
Entity: நல்ல, Label: B-COMM, Score: 0.6417
Entity: வியாபார, Label: B-COMM, Score: 0.7069
Entity: ிகள், Label: B-COMM, Score: 0.7750


In [ ]:
def aggregate_entities(ner_results):
    aggregated_entities = []
    current_entity = None

    for entity in ner_results:
        word, label, score = entity["word"], entity["entity_group"], entity["score"]

        if current_entity and label == current_entity["label"] and label in ["PER", "ORG", "COMM"]:
            # Merge consecutive entity words of the same type
            current_entity["word"] += word
            current_entity["score"] = (current_entity["score"] + score) / 2
        else:
            if current_entity:
                aggregated_entities.append(current_entity)
            current_entity = {"word": word, "label": label, "score": score}

    if current_entity:
        aggregated_entities.append(current_entity)

    return aggregated_entities

# text = "லால் காந்த மக்கள் விடுதலை முன்னணி பதவி விலகல் ரம்புகன விதான பதிரண சரத் எம் . கணேஷன் ஜனநாயக மக்கள் முன்னணி பாராளுமன்றத்திற்குத் தெரிவு செய்யப்பட்டமை  "
text = "முஸ்லிம்கள் இந்த நாட்டில் மிகவும் நல்ல வியாபாரிகள்"
ner_results = ner_pipeline(text)
ner_results = aggregate_entities(ner_results)

# Display results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['label']}, Score: {entity['score']:.4f}")


Entity: முஸ்லிம்கள்மிகவும்நல்லவியாபாரிகள், Label: COMM, Score: 0.6974


In [ ]:
tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True)
print(tokenizer.convert_ids_to_tokens(tokens["input_ids"][0]))


['<s>', '▁முஸ்லிம்', 'கள்', '▁இந்த', '▁நாட்டில்', '▁மிகவும்', '▁நல்ல', '▁வியாபார', 'ிகள்', '</s>']


In [ ]:
def merge_entities(ner_results, tokens):
    merged_results = []
    current_entity = None

    for i, entity in enumerate(ner_results):
        word, label, score = entity["word"], entity["entity_group"], entity["score"]

        # Check if subword (does not start with ▁)
        is_subword = not tokens[i].startswith("▁")

        # Ignore words that should not be part of an entity
        if label == "COMM" and word in ["மிகவும்", "நல்ல"]:
            continue  # Skip unwanted words

        if current_entity and label == current_entity["label"] and is_subword:
            current_entity["word"] += word  # Merge subword
            current_entity["score"] = (current_entity["score"] + score) / 2  # Average confidence
        else:
            if current_entity:
                merged_results.append(current_entity)
            current_entity = {"word": word, "label": label, "score": score}

    if current_entity:
        merged_results.append(current_entity)

    return merged_results


In [ ]:
tokens = tokenizer(text.split(), return_offsets_mapping=True, return_tensors="pt", is_split_into_words=True)


In [ ]:
text = "முஸ்லிம்கள் இந்த நாட்டில் மிகவும் நல்ல வியாபாரிகள்"

tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True)
ner_results = ner_pipeline(text)

tokens_list = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])  # Get tokenized words
ner_results = merge_entities(ner_results, tokens_list)

# Display results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['label']}, Score: {entity['score']:.4f}")


Entity: முஸ்லிம்கள், Label: COMM, Score: 0.9044
Entity: வியாபாரிகள், Label: COMM, Score: 0.7069
